<a href="https://colab.research.google.com/github/AtharKharal/Data_science_with_python/blob/dev-branch/featuretools_and_TPOT_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import featuretools as ft
import sklearn as sk

#Reading Data

##From a Local Drive

In [0]:
#from google.colab import files
#uploaded = files.upload()

#import io
#df2 = pd.read_csv(io.BytesIO(uploaded['Filename.csv']))

##From Google Drive via PyDrive

In [10]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 18.8MB/s 


In [0]:
# Go to the CSV file in Google Drive, right-click on it and select 
# “Get shareable link”. The link will be copied into your clipboard. 
# Paste this link into following a string variable (say) link:

link = 'https://drive.google.com/open?id=1U6CRwCxBR4_GxG5kig-mHeet1kykX6Sk'  # The shareable link

fluff, id = link.split('=')   # Verify that you have everything after '=' by using print(id)

# Finally reading the data into a dataframe

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dummy_file_name.csv')  
df = pd.read_csv('dummy_file_name.csv')

In [20]:
df.head()

,CadetPakNo,OffPakNo,Rank,Name,RegNoAU,MatricPercentage,FScPercentage,LastAttendedInstitute,CNIC_No,DateOfBirth,...,Stream,Semester,CurrentCourse,DateOfGraduation,DateCompletionBS,AcademicPercentage,ProfessionalPercentage,DisciplinePercentage,DataQuery,Remarks
0,97511.0,16645.0,Plt Off,Muhammad Bilal Zulqarnain,AU-130054,79.65,68.55,"PAF Public School LOWER TOPA, Murree",36502-6719547-7,18-12-1990,...,ATC,Completed,17 A&SD,10/6/2016,8/24/2017,87.58,86.57,70.14,NaN,NaN
1,1013120.0,16791.0,Flg Off,Abaidullah Mir,AU-100020,89.41,81.36,PAF Public School SARGODHA,34101-0608923-5,4/8/1992,...,Tpt,Completed,7th CN-235,3/26/2015,8/24/2017,72.05,78.17,59.58,NaN,NaN
2,107616.0,16832.0,Plt Off,Waqas Khattak,AU-140056,74.38,75.19,New Fazaia Degree College Peshawar Cantt,17301-76106955,31-10-1988,...,Nav,VII,5th Nav,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11132012.0,16936.0,Flg Off,Ali Qasim,AU-110012,87.88,79.00,"Fauji Foundation College for Boys, New Lalazar...",37405-9164177-7,28-02-1992,...,Fighter,Completed,134 GD(P),10/14/2015,4/4/2017,81.77,82.16,74.19,NaN,Late due to Course Abroad USA
4,11132007.0,16947.0,Flg Off,Salik Anns,AU-110007,74.59,66.00,Punjab College of Science Gujranwala,34101-4944832-1,2/2/1993,...,Fighter,Completed,133 GD(P),3/26/2015,8/24/2017,70.64,78.81,67.31,NaN,NaN


#Feature Augmentation

In [0]:
es = ft.EntitySet("mydata")

# Create an entity from the client dataframe
# This dataframe already has an index and a time index col namely 'dt_time_col'
es = es.entity_from_dataframe(entity_id = 'my_df_id', dataframe = my_df, 
                              index = 'idx_of_my_df', time_index = 'dt_time_col')


# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='my_df_id', 
                                 max_depth = 2)

#Pipeline Optimization

In [0]:
feature_matrix = pd.read_csv('./example_data.csv', index_col=0)


X = feature_matrix.drop('MEAN(invoices.MEAN(item_purchases.UnitPrice))', axis=1)
y = feature_matrix['MEAN(invoices.MEAN(item_purchases.UnitPrice))']


X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42)

tpot.fit(X_train, y_train)


y_hat = tpot.predict(X_test)
print("R2 score:", sklearn.metrics.r2_score(y_test, y_hat))

In [0]:
tpot.export('./tpot_pipeline.py')

#from the python file 'tpot_pipeline.py' copy following two lines verbatim

training_features, testing_features, training_target, testing_target = \
    train_test_split(features, tpot_data['class'], random_state=42)

exported_pipeline = make_pipeline(
    RobustScaler(),
    LinearSVR(C=15.0, dual=False, epsilon=1.0, 
              loss="squared_epsilon_insensitive", tol=0.0001)
)


exported_pipeline.fit(training_features, training_target)


In [0]:
#Show the features with each used coefficient:

important_coefs = pd.Series(data=exported_pipeline.steps[1][1].coef_, 
                            index=X.columns)

sorted_coef = important_coefs.sort_values(ascending=False)

sorted_coef